In [1]:
#Creating training and test files from given data
import csv

rownames=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,"Class"]
with open('satimage_training.csv', 'w+') as csvFile:
    
    writer = csv.writer(csvFile)
    writer.writerow(rownames)
    for contents in open("satimage.scale.training","r"):
    #for j in range(1):
        row=[]
        #f= open("satimage.scale.training","r")
        #contents =f.readline()
        line=contents.split()
        read=1;
        for i in range(1,37): #f.mode == 'r':
            #print(line[i].split(":"))
            if (int(line[read].split(":")[0]))!=i:
                row.append("")
                #print(int(line[read].split(":")[0]),":",i)
                continue
            row.append(line[read].split(":")[1])
            read=read+1
        row.append(line[0])
        #print(row)
        writer.writerow(row)
csvFile.close()

In [2]:
with open('satimage_test.csv', 'w+') as csvFile:
    
    writer = csv.writer(csvFile)
    writer.writerow(rownames)
    for contents in open("satimage.scale.t","r"):
    #for j in range(1):
        row=[]
        #f= open("satimage.scale.training","r")
        #contents =f.readline()
        line=contents.split()
        read=1;
        for i in range(1,37): #f.mode == 'r':
            #print(line[i].split(":"))
            if (int(line[read].split(":")[0]))!=i:
                row.append("")
                #print(int(line[read].split(":")[0]),":",i)
                continue
            row.append(line[read].split(":")[1])
            read=read+1
        row.append(line[0])
        #print(row)
        writer.writerow(row)
csvFile.close()

In [3]:
import pandas as pd

df = pd.read_csv('satimage_training.csv')
df.dropna(inplace=True)
X_train = df[['1','2','3','4','5','6','7','8','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36']]
y_train = df[['Class']]
df = pd.read_csv('satimage_test.csv')
df.dropna(inplace=True)
X_test = df[['1','2','3','4','5','6','7','8','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36']]
y_test = df[['Class']]
Class_dict = {
    1:0,2:0,3:0,4:0,5:0,
    6:1
}
y_train=y_train.Class.map( Class_dict )
y_train=y_train.to_frame()
y_test=y_test.Class.map( Class_dict )
y_test=y_test.to_frame()

In [4]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import os

rslt_addr = "./results/Bdlaskndsal/"
rslt_addr = "./results/"
if not os.path.exists(rslt_addr):
    os.makedirs(rslt_addr)
X_train = pd.DataFrame(normalize(X_train))
X_test = pd.DataFrame(normalize(X_test))
n_folds = 10
c_vals = np.power(float(10), range(-7, 10 + 1))

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

def cross_validate_c_vals(X, y, c_vals, n_folds,d):
    
    c_auc_dict = {}
    c_auc_folds = {}
    support_vectors = 0
    for c in c_vals:
        c_auc_dict[c] = 0
        c_auc_folds[c] = []
        skf=StratifiedKFold(n_splits=n_folds, shuffle=True)
        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index].values.ravel(), y.iloc[test_index].values.ravel()

            svm = SVC(C=c, kernel="poly", degree=d, shrinking=False)
            
            svm.fit(X_train, y_train)
            
            support_vectors = support_vectors + (svm.support_vectors_[0] + svm.support_vectors_[1])/n_folds
            
            pred   =   svm.predict(X_test)
            
            c_auc_folds[c].append(roc_auc_score(y_test, pred))
            c_auc_dict[c]   += roc_auc_score(y_test, pred)/n_folds
        
    return c_auc_dict,c_auc_folds,support_vectors

In [6]:
import os
import warnings
warnings.simplefilter('ignore')

#rslt_addr = "./results/Bdlaskndsal/"
rslt_addr = "./results/"
if not os.path.exists(rslt_addr):
    os.makedirs(rslt_addr)
    
rslt_dict_name = "svm_results.pickle"
rslt_dict_addr = os.path.join(rslt_addr, rslt_dict_name)

import numpy as np
import pickle

c_vals = np.power(float(10), range(-10, 9 + 1))
n_folds = 10
d_support_vectors={}
d_dict={}
d_dict_fold={}

for d in range(1,5):
    c_auc_dict,c_auc_folds,support_vectors = cross_validate_c_vals(X_train, y_train, c_vals, n_folds,d)
    d_support_vectors[d]=support_vectors
    d_dict[d]=c_auc_dict
    d_dict_fold[d]=c_auc_folds

KeyboardInterrupt: 

In [ ]:
m=1
d_optim={}
for d in range(1,5):
    m=1
    for c in c_vals:
        if m==1:
            maximum=d_dict[d][c]
            optim_c=c
            m==2
        if d_dict[d][c]>maximum:
            maximum=d_dict[d][c]
            optim_c=c
    d_optim[d]=optim_c

In [ ]:
for d in range(1,2):
    c=d_optim[d]
    svm = SVC(C=c, kernel="poly", degree=d, shrinking=False)
    svm.fit(X_train, y_train)
    pred = svm.predict(X_test)
    